# Sea ice autocorrelation

In [ ]:
# Input

path="C:/Users/slibera/data/NSIDC/sic_combined.nc"
maxlag=13
Nyear=29
xMon=0

### Setting data

The data is loaded, set it to be consistent for COSIMA 0.1 deg year ranges, avoid NaN values. 

Monthly detrended time series are created

Function for calculating the correlation is also defined

In [27]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt 
import scipy.stats
import scipy.signal

################################### SETTING DATA ################################################

data=xr.open_dataset(path)                      #Original data, 
sic=data.goddard_merged_seaice_conc_monthly     #Dimensions :(time: 469, ygrid: 332, xgrid: 316)
sic                                             # 3 coordinates, ygrid, xgrid, time (*all callable)

# Making the years consisternt with COSIMA 0.1 deg model and dropping NaN years (1987,1988)

sic=sic[2:,...]
sic=sic.where(sic['time.year']!=1987,drop=True) #Drop year 1987
sic=sic.where(sic['time.year']!=1988,drop=True) #Drop year 1988 [Avoiding the computations with nan]
for i in range (1978,1985):
    sic
    sic2=sic.where(sic['time.year']!=i,drop=True)
    sic=sic2

    
sic=sic.where(sic['time.year']!=2017,drop=True) #Drop year 2017
sic=sic.where(sic['time.year']!=2018,drop=True) #Drop year 2018    

sic=sic[1:349,:,:] #348 = Jan 2016
#sic

# Creating individual monthly time series using stride

feb=sic[::12,:,:]
mar=sic[1::12,:,:]
apr=sic[2::12,:,:]
may=sic[3::12,:,:]
jun=sic[4::12,:,:]
jul=sic[5::12,:,:]
aug=sic[6::12,:,:]
sep=sic[7::12,:,:]
oct=sic[8::12,:,:]
nov=sic[9::12,:,:]
dec=sic[10::12,:,:]
jan=sic[11::12,:,:]

# Detrending monthly timeseries and creating new dataset

months=[jan, feb, mar, apr, may, jun, jul, aug, sep, oct, nov, dec]
d_months=months

for i in range(12): 
    new_dims=months[i].dims[:]
    new_coords=months[i].coords
    detrend=scipy.signal.detrend(months[i], axis=0)
    d_months[i]=xr.DataArray(detrend.squeeze(),dims=new_dims,coords=new_coords)
    d_months[i]=d_months[i].stack(z=('ygrid','xgrid'))

new_ds=d_months  
# The new dataset is a list and not a concatenated array as Xarray would otherwise fill the change in year using NaN
# Also, (.drop(time)) is not applied here for avoiding this as numpy.datetime64 is utilized to identify change in start year


# Function for calculating correlation using scipy.stats.pearsonr
# Modified for the input dataset
    
def corr(ar,axis):
    '''Correlation coefficient between 2 timeseries concatenated together
    ar: 2D array, time is the right dimension. Left dimension must be size 2. 
        ar[0,:] is the first timeseries
        ar[1,:] is the second timeseries
    stats: 1D array containing the correlation coefficient and the p-value.'''
    # Remove any degenerated dimensions
    ar1=ar[0,...].squeeze()
    ar2=ar[1,...].squeeze()
    stats = xr.DataArray(np.asarray(scipy.stats.pearsonr(ar1,ar2)),dims='c')
    stats = stats.expand_dims(dim='z',axis=1)
    return stats

dummy_dat=d_months[0]
#dummy_dat created only to specify dimensions in the 'out_ar' array created to store the correlation results



<xarray.DataArray 'goddard_merged_seaice_conc_monthly' (time: 348, ygrid: 332, xgrid: 316)>
array([[[0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.],
        ...,
        [0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.]],

       [[0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.],
        ...,
        [0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.]],

       ...,

       [[0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.],
        ...,
        [0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.]],

       [[0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.],
        ...,
        [0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.]]], dtype=float32)
Coordinates:
  * ygrid      (ygrid) float32 4337500.0 4312500.0 ... -3912500.0 -3937500.0
  * xgrid      (xgrid) float32 -3937500.0 -3912500.0 ... 3912500.0 3937500.0
    latitude   (ygrid, xgrid) float64 -39.36 -39.49 -39.62 ... -41.72 -41.58
    longitude  (ygrid, xgrid) float64 -42.23 -42.05 -41.87 ... 135.4 135.2 135.0

### Correlation Calculation

For consistent year ranges, only consideration is the year change with increasing lags.
That is, shift the start year for the lag month and reduce the end year for the lead month for consistent length of datasets.

When the year ranges are changing, we need to consider multiple cases. 

Sample scenario: Data from September 2000 to August 2011.
This dataset can be further divided to monthly timeseries with two set of year ranges:    
January to August [2001-2011]; September to December [2000-2010]
    
(1) When lead month starts later [without change in year, ie (xMon+lag)<11]  
    Eg Lead: March[2001-2011] Lag: October[2000-2010]
    
(2) When lead month starts later and the year change is also in place 
    Eg Lead: March[2001-2011] lag: February[2001-2011] ----- (Normal scenario!)
            
(3) When lag month starts later [with year change]   
    Eg Lead: October[2000-2010] lag: Febrary[2001-2011]-----(Plus additional shift in start year)
            
For the possible cases of inconsistent year ranges for the leads and lags and for the additional affect of year change, 
we need to apply more conditions.

In the calculation we choose the Lead and Lag month timeseries and identify the start year, then according to the lag we apply
conditions

In [31]:
xMon=3


# out_ar - array to store correlation outputs

out_ar = xr.DataArray(np.zeros((maxlag-1,2,dummy_dat.shape[-1])),dims=('lag_n','acf','z'),  #maxlag-1 change to maxlag
                     coords={'acf':['r','p-value'],
                             'lag_n':list(range(1,maxlag)),
                             'z':dummy_dat.z})


for lag in range(1,maxlag):
    yMon  = lag+xMon
    count = yMon                         # To consider year change later
    #print(xMon, yMon)
    if yMon>11:                          #if the considered month(y) is in the upcoming year to month(x)
        yMon   = yMon-12
        xEn_0  = -1                      # Change range of years in correlation X:(0-end_year-1)
        ySt_0  = 1                       # Change range of years in correlation Y:(1-end_year)
        yEn_0  = 0
    else:
        ySt_0  = 0
        xEn_0  = 0
        xSt_0  = 0
#------------------------------------------ Considering changes in year        
    x_dash=new_ds[xMon]
    y_dash=new_ds[yMon]
    xMon_start=x_dash.time.dt.year[0]    # Starting year for the lead month
    yMon_start=y_dash.time.dt.year[0]    # Starting year for the lag month
#-------------------------------------------------------------------        
    if xMon_start > yMon_start:          # When lead month starts later (Here, if lead month was January )  
        ySt_1 = 1                        # January [1986-2016]; other months [1985-2015] 
        xEn_1 = -1
    else:                                # Change period like: January [1986-2015]; other months [1986-2015]--> ySt+1, xEn-1
        ySt_1 = 0                        # Have to consider YEAR CHANGE (if (xMon_start > yMon_start) and (count < 11):)
        xEn_1 = 0        
#-----------------------------------------------------------------------         
        
    if yMon_start > xMon_start:          # When lag month starts earlier
        xSt_2 = 1                        # Eg: lead= Nov[1985-2015]; lag= Jan[1986-2016]
        yEn_2 = -1                       # Change period Nov[1986-2015]; lag[1986-2015]--> xSt+1, yEn-1 
    else:
        xSt_2 = 0
        yEn_2 = 0 
#------------------------------------------------------------------------        
        
    if (yMon_start > xMon_start) and (count>11):
        ySt_3 = -1                       # Considering year change with lag month starting later
    else:                                # with year change we would shift year again [when yMon>11: ySt=1]
        ySt_3 = 0                        # For Jan[1986-2016]; we would shift it to Jan[1987-2016]; So we have to offset that change
   
#
#------------------------------------------------------------------------------------------------------------------
# Finally, calculating the lead and lag ranges and selecting dataset for calculation

    xSt = xSt_0 + xSt_2
    xEn = Nyear - (xEn_0 + xEn_1)
    ySt = ySt_0 + ySt_1 + ySt_3
    yEn = Nyear - (yEn_0 + yEn_2)
    
    X=x_dash[xSt:xEn,:].drop('time')    # Here time is dropped to avoid xarray concat from creating intermediate years with fill values 
    Y=y_dash[ySt:yEn,:].drop('time') 

# Printing out range and values for testing
    
    print('Lag=',lag,'-------','yMon=',yMon)
    print (yEn_0 + yEn_2)
    print('xMon:', xMon,'(',xSt,':',xEn,')','-----','yMon:',yMon,'(',ySt,':',yEn,')' )
    print(yMon_start)    

#----------------------------------- Calculation ----------------------------------------------    

# Commented out while testing

    #xy=xr.concat([X,Y])

    #cor23=xy.groupby('z').reduce(corr,dim='time') # CORRELATION CALCULATION

    #cor23.rename(concat_dims='stats')             #renaming a dimension in the result
    #out_ar[lag-1,...]=cor23                       # Writing result to out_ar 
    #out_test=cor23.unstack()
    
        

Lag= 1 ------- yMon= 4
0
xMon: 3 ( 0 : 28 ) ----- yMon: 4 ( 0 : 28 )
<xarray.DataArray 'year' ()>
array(1985, dtype=int64)
Coordinates:
    time     datetime64[ns] 1985-05-01
Lag= 2 ------- yMon= 5
0
xMon: 3 ( 0 : 28 ) ----- yMon: 5 ( 0 : 28 )
<xarray.DataArray 'year' ()>
array(1985, dtype=int64)
Coordinates:
    time     datetime64[ns] 1985-06-01
Lag= 3 ------- yMon= 6
0
xMon: 3 ( 0 : 28 ) ----- yMon: 6 ( 0 : 28 )
<xarray.DataArray 'year' ()>
array(1985, dtype=int64)
Coordinates:
    time     datetime64[ns] 1985-07-01
Lag= 4 ------- yMon= 7
0
xMon: 3 ( 0 : 28 ) ----- yMon: 7 ( 0 : 28 )
<xarray.DataArray 'year' ()>
array(1985, dtype=int64)
Coordinates:
    time     datetime64[ns] 1985-08-01
Lag= 5 ------- yMon= 8
0
xMon: 3 ( 0 : 28 ) ----- yMon: 8 ( 0 : 28 )
<xarray.DataArray 'year' ()>
array(1985, dtype=int64)
Coordinates:
    time     datetime64[ns] 1985-09-01
Lag= 6 ------- yMon= 9
0
xMon: 3 ( 0 : 28 ) ----- yMon: 9 ( 0 : 28 )
<xarray.DataArray 'year' ()>
array(1985, dtype=int64)
Co

out_ar = xr.DataArray(np.zeros((maxlag-1,2,dummy_dat.shape[-1])),dims=('lag_n','acf','z'),  #maxlag-1 change to maxlag
                     coords={'acf':['r','p-value'],
                             'lag_n':list(range(1,maxlag)),
                             'z':dummy_dat.z})


for lag in range(1,maxlag):
    yMon = lag+xMon
    if yMon>11:            #if the considered month(y) is in the upcoming year to month(x)
        yMon = yMon-12
        
        
        
        xEn  = Nyear-1     # Change range of years in correlation X:(0-end_year-1)
        ySt  = 1           # Change range of years in correlation Y:(1-end_year)
        yEn  = Nyear
    else:
        ySt  = 0
        xEn  = Nyear
        
    x_dash=new_ds[xMon]
    y_dash=new_ds[yMon]
    
    xMon_start=x_dash.time[0]
    yMon_start=y_dash.time[0]
    
    if xMon_start>yMon_start:
        xEn=xEn-1
        ySt=ySt
        yEn=Nyear-1
    X=x_dash[:xEn,:]
    Y=y_dash[ySt:Nyear-1,:]
    yMon_start2=Y.time[0]
    xMon_end=x_dash.time[-xEn]
    yMon_end=y_dash.time[-1]
    print(yMon, xEn, ySt)
    print(yMon_start2)
    print(xMon_start)
    

        
        xEn  = Nyear-1     # Change range of years in correlation X:(0-end_year-1)
        ySt  = 1           # Change range of years in correlation Y:(1-end_year)
        yEn  = Nyear
    else:
        ySt  = 0
        xEn  = Nyear
        
    

    
    
    
        xEn=xEn-1
        ySt=ySt
        yEn=Nyear-1
    X=x_dash[:xEn,:]
    Y=y_dash[ySt:Nyear-1,:]
    yMon_start2=Y.time[0]
    xMon_end=x_dash.time[-xEn]
    yMon_end=y_dash.time[-1]
    print(yMon, xEn, ySt)
    print(yMon_start2)
    print(xMon_start)
    